In [113]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
from datetime import datetime
from copy import deepcopy
load_dotenv()

True

## load variables - contours
- data folder
- cleaned_df - for only contours
- json template
- 

In [119]:
cd ../

/Users/victor/Documents/projects/mini_projects/regreen_cleaning/brics3_migration


In [121]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
dff = pd.read_csv(cleaned_csv)

#lambda function to filter for contours
#for each row, is it a list/str, then is 'contours' in it, if so, return the df.
cleaned_df = dff[
    dff["land_use_selection-intervention"]
    .apply(lambda x: isinstance(x, (list, str)) and "contour_planting" in x)
]

#contours datasets
contours_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'outputs','cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_contour.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")
with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



## check contours count

In [122]:
x = dff["land_use_selection-intervention"]
count = 0
for idx, i in enumerate(list(x)):
    if 'contour_planting' in i:
        count += 1
        #ic(i, count)

ic('contours count',len(cleaned_df))

ic| 'contours count', len(cleaned_df): 9


('contours count', 9)

## enumerators

In [150]:
#sorted(list(dff['basic_info-name'].unique()))
##empty name and key dictionary
names_dict = {}
for idx, row in dff.iterrows():
    #ic({row["KEY"], row["basic_info-name"]})
    names_dict[row["KEY"]] = row["basic_info-name"]
#ic(names_dict)
name_uname_dict = {
    "Zakariye haji ali": "abdullahi.hassan@mardo.org", 
    "Issack Idow Ibrahim": "abdullahi.hassan@mardo.org",
    "Abdullahi Hassan Adan": "abdullahi.hassan@mardo.org",
    "Abdikheir Mohamed": "mukhtar.yusuf23",
    "Abdikheir Mohamed Ali": "mukhtar.yusuf23",
    "Abdikheir Mohamed Dahir": "mukhtar.yusuf23",
    "Ali": "mukhtar.yusuf23",
    "Ali Adan": "mukhtar.yusuf23",
    "Ali Aden": "mukhtar.yusuf23",
    "Fathi": "mukhtar.yusuf23", 
    "Fathi Abdirashid": "mukhtar.yusuf23",
    "Ahmed Omar Hassan": "AhmedOmar",
    "Maria Abdirahman Ali": "AhmedOmar",
    "ABSHIR ABDULLAH ALI": "Ahil",
    "Ahmed Ibrahim": "JubaFoundation",
    "Ahmed Ibrahim Mohamed": "JubaFoundation",
    "Abdiaziz Adan Hassa": "guudow",
    "Abdiaziz Adan Hassan": "guudow",
    "Abdi Hassan Adan": "AbdiHassan",
    "Abdullahi": "sharif1234",
    "Abdirashid Sheikh Mohamed": "NRMO",
    "Abdullahi Sharif": "sharif1234",
    "Abdullahi Sharif LLG": "sharif1234",
    "Abdullahi Sharif Noor LLG": "sharif1234",
    "Abdijalil": "Csxaashi",
    "Abdijalil said": "Csxaashi",
    "Abdikarim Mohamed Ismail": "samandoulgou",
    "Abdikarin Mohamed Ismail": "samandoulgou",
    "Hassan Omar Khadhib": "Khadhib",
    "Hassan Omar Khadhibr": "Khadhib",
    "Jamal Ali": "JamalAli",
    "Jamal Ali Tagal": "JamalAli",
    "Mohamud Abdullahi Ali": "Mohamud",
    "Moulid Abdullahi Abdi": "Moulidwadani",
    "Muktar Mohamednoor Rage": "Mrage1997",
    "Osman Ebey Omar": "iibey",
    "Yusuf Mahi munye": "Mcnbdn617660",
    "abdullahi Sharif LLG": "sharif1234",
    "osman Ebey Omar": "iibey",
}

## match and replace names with usernames
for k, v in names_dict.items():
    for key, value in name_uname_dict.items():
        if v == key:
            names_dict[k] = value
#ic(names_dict)

##enumerator dictionary
enumerator = {
        "id": 3726,
        "first_name": "First",
        "last_name": "Last",
        "gender": "Male",
        "age_category": "18 - 35",
        "phone_number": "0e15704046",
        "email": "test_mail@gmail.com",
        "country": "test",
        "organization": "Test",
        "username": "test",
        "type": "ENUMERATOR"
    }

enums = []
#create enumerator jsons
for key, value in names_dict.items():
    e = deepcopy(enumerator)
    e['username'] = value
    e['key'] = key
    enums.append(e)

#ic(enums)

## farming entity

In [151]:
farming_entity_list = []
for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    farming_entity = {}
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    farming_entity['key'] = i['KEY']
    
    farming_entity_list.append(
        {
            "farmingEntity": farming_entity, 
            'key': i['KEY']
        }
    )
    #ic(farming_entity_list)
    

## plot-contours_details
- read df to dictionary, for matching keys, extract df values
### plot-points
- parse the geoshape column from cleaned df
- split and extract the values from each i
- append to json keys

## crops
- get matching rows based on key(id)
- for these rows if crop has crops, get its values and 'other' if any
    - every matching rows values are appended to a list(of all crops for that plot)
- clean 'and' from the crop types



In [152]:
import uuid
import copy
import re

plot_list = []
point_id = 1

records = cleaned_df.to_dict(orient='records')

count = 0
#iterate over every i to capture plot details
for idx, i in enumerate(records):
    plot_id = idx + 1
    count += 1

    # ---- BUILD PLOT DETAILS ----
    template_cp = copy.deepcopy(template[0])
    plot_details = template_cp['plot']

    plot_details['id'] = plot_id
    plot_details['key'] = i['KEY']
    plot_details['name'] = uuid.uuid4().hex
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']
    plot_details['livestock_allowed'] = False if i['land_use_selection-land_use'] == 'agriculture' else True
    plot_details['conservation_area'] = False

    #get only child row for this key
    matching_child_rows = contours_df[contours_df['PARENT_KEY'] == i['KEY']]

    #holds every row's crops
    all_crops = []
    """----------------------------------CROP STATUS ----------------------------------"""
    #iterate over matching child is to build crop status
    for x in pd.DataFrame(matching_child_rows).to_dict(orient='records'):
        #ic(x['PARENT_KEY'], x["KEY"], x["managements_contourcrops_in_field2"], x["managements_contour-other_crops"])

        if x['managements_contour-crops_present5'] == 'no' or pd.isna(x['managements_contour-crops_present5']):
            plot_details['has_crops'] = False
        elif x['managements_contour-crops_present5'] == 'yes':
            plot_details['has_crops'] = True


        #get a list of all crops in field
        if pd.isna(x['managements_contour-crops_in_field5']):
            crop_list = []
        else:
            crop_list = str(x['managements_contour-crops_in_field5']).split(" ")
            ic(crop_list, x['KEY'])


        #check that 'other' crops exist
        if 'other' in crop_list:
            #split by comma or 'and'
            other_crops_value = re.split(r',|\band\b', x['managements_contour-other_crops'])
            #strip whitespace
            other_crops_value = [c.strip() for c in other_crops_value if c.strip()]

            #get rid of 'and' if exists
            if 'and' in other_crops_value:
                other_crops_value.remove('and')

            #only add if not empty
            if other_crops_value != []:
                crop_list.remove('other')
                crop_list.extend(other_crops_value)
                #append crop list for this row to all crops
            all_crops.extend(crop_list)
            ic(all_crops)
        
        elif 'other' not in crop_list:
            all_crops.extend(crop_list)

        plot_details['crops'] = list(set(all_crops))

        
    """--------------------------------GEOSHAPE PARSING ----------------------------------"""
    geoshape_str = i['plot_details-polygon']
    
    def parse_geoshape(geoshape_str):
        #parse a geoshape string into a list of point dictionaries
        points = []
        parts = geoshape_str.split(';')

        for part in parts:
            part = part.strip()
            if not part:
                continue

            lat, lon, alt, acc = map(float, part.split(' '))

            points.append({
                "longitude": lon,
                "latitude": lat,
                "altitude": alt,
                "accuracy": acc
            })

        return points

    parsed_points = parse_geoshape(geoshape_str)

    points_list = []
    for p in parsed_points:
        points_list.append({
            "id": point_id,
            "plotId": plot_id,
            "longitude": p["longitude"],
            "latitude": p["latitude"],
            "altitude": p["altitude"],
            "accuracy": p["accuracy"]
        })
        point_id += 1


    """--------------------------------ADMINISTRATIVE DETAILS ----------------------------------"""
    administrative = plot_details['subCounty']
    administrative['subcounty_name'] = i['geography-district_name']
    administrative['county']['county_name'] = i['geography-region_name']


    #update plot_details with number of points and administrative info
    plot_details['points'] = points_list
    plot_details['subCounty'] = administrative
    
    # del plot_details['points']
    # del plot_details['subCounty']
    general_plot = {
        "plot":plot_details, 
        'key':i['KEY']
    }
    #ic(i['KEY'])
    plot_list.append(
        general_plot
    )


ic| crop_list: ['sugarcane', 'maize', 'sorghum', 'beans', 'banana', 'date']
    x['KEY']: 'uuid:c9db5e91-ee4d-4098-92ca-1551e55a6c28/contour/trees_grasses_contour[1]'
ic| crop_list: ['sugarcane', 'maize', 'beans', 'banana', 'orange', 'date']
    x['KEY']: 'uuid:c9db5e91-ee4d-4098-92ca-1551e55a6c28/contour/trees_grasses_contour[2]'
ic| crop_list: ['maize', 'sugarcane', 'banana', 'date', 'tomato']
    x['KEY']: 'uuid:c9db5e91-ee4d-4098-92ca-1551e55a6c28/contour/trees_grasses_contour[3]'


In [153]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(plot_list, f, indent=4)

str(contours_df['managements_contour-crops_in_field5'][0]).split(' ')

['nan']

# project

In [154]:
projects_list = []

names =[]
for row in cleaned_df.to_dict(orient='records'):
    name = row['land_use_selection-project_name']
    names.extend([name])

    if name == 'terra':
        name = name.upper()
        id = 131
    elif name == 'brcis3':
        name = 'BRCiS III'
        id = 93

    projects_list.append({
        "project": {
            "id": id,
            "project_name": name
        },
        "key": row['KEY']
    })

ic(set(names))

ic| set(names): {'RAF', 'brcis3'}


{'RAF', 'brcis3'}

## contours 
missing 
- herbacious cover
- who manages >> used who established
- erosion type
- is_intervention_effective
- vertical_spacing
- horizontal_spacing

In [155]:
contours_list = []

#dictionary to map odk values to RA app schema values
mapping_dict = {
    "STICKS_BRANCH":"STICKS_PROTECTION",
    "DEBRIS_REMOVAL":"DEBRIS_REMOVAL",
    "INVASIVE_SPECIES":"INVASIVE_CONTROL",
    "WEEDING":"WEEDING",
    "WATERING":"WATERING",
    "PLANTING_SEEDLING":"PLANTING_ANNUAL_PERENNIAL_CROPS"
}


for idx, c in enumerate(records):

    cureent_status_list = []
    all_grasses = []
    all_trees = []

    #if c['KEY'] == "uuid:1975a725-83a5-4bdd-8305-35ac65b2cce9":
    """-----------------------------------contours CHILD DETAILS-------------------------------------"""

    total_num_grasses_planted = 0
    total_kg_grass_seeds = 0.0
    total_num_tree_species = 0
    total_num_trees_planted = 0
    total_num_trees_survived = 0
    grass_present = []
    trees_present = []
    mngmts = []
    
    #for every record, get the matching child rows
    matching_child_rows = contours_df[contours_df['PARENT_KEY'] == c['KEY']]
    #matching_child_rows = contours_df[contours_df['PARENT_KEY'] == "uuid:91755b0a-6073-4d6d-88a8-55eaa382d3e6"]
    idxx =0
    for x in matching_child_rows.to_dict(orient='records'):

        total_num_grasses_planted += 0 if pd.isna(x["grass_contour-num_grass_species"]) else x["grass_contour-num_grass_species"]
        total_kg_grass_seeds += 0 if pd.isna(x["grass_contour-kg_grass_seeds"]) else x["grass_contour-kg_grass_seeds"]
        total_num_tree_species += 0 if pd.isna(x["tree_contour-num_tree_species"]) else x["tree_contour-num_tree_species"]
        total_num_trees_planted += 0 if pd.isna(x["tree_contour-num_trees_planted"]) else x["tree_contour-num_trees_planted"]
        total_num_trees_survived += 0 if pd.isna(x["tree_contour-num_tree_survived"]) else x["tree_contour-num_tree_survived"]
        grass_present.append(True if str(x["tree_contour-planted_choice5"]).strip() in ["both", "grasses"] else False)
        trees_present.append(True if str(x["tree_contour-planted_choice5"]).strip() in ["both", "trees"] else False)
        mngmts.extend(str(x["managements_contour-management_practices"]).upper().split()) if not pd.isna(x["managements_contour-management_practices"]) else 'NAN'

        """--------------TREE ESTABLISHMENT-------------"""
        tree_status_list = []
        if x['tree_contour-planted_choice5'] in ["both", "trees"]:
            trees_species_list = x["tree_contour-contour_trees_species"].split()
            ic(trees_species_list)
            j = 0 
            for idx3, raw in enumerate(trees_species_list):
                #ic("original",raw)
                #default
                scientific_name = ""
                local_name = ""
                
                #case1: check or other in the tree species list
                if raw == 'other':
                    #ic("other",raw)
                    other_val = str(x["tree_contour-other_contour_trees"]).strip()
                    #ic("other_val", other_val)

                    #split by comma & 'and'
                    names = re.split(r",| and ", other_val, flags=re.IGNORECASE)
                    #ic(names)
                    for idxx, val in enumerate(names):
                        local_name = val.strip()
                        #handle nan cases
                        if local_name != 'nan' and local_name != "":
                            k += 1
                            tree_current_status = {
                                "id": idxx + 1,
                                "local_name": local_name.capitalize(),
                                "scientific_name": scientific_name,
                                "econtrol_status_id": 1
                            }
                            ic(tree_current_status)
                            tree_status_list.append(tree_current_status)

                #case 2: scientific + local name extraction
                if "(" in raw and ")" in raw:
                    #scientific name cleaning
                    parts = raw.split("(")[0].strip().replace("_", " ").split()
                    #ic(parts)
                    if len(parts) >= 2:
                        scientific_name = f"{parts[0].capitalize()} {parts[1].lower()}"
                        #ic(scientific_name)

                    #local name cleaning
                    paarts = raw.split("(")[1].split(")")[0].strip().replace("_", " ").replace(",", " ").split()
                    #ic(paarts)
                    if len(paarts) >= 2:
                        local_name = f"{paarts[0].capitalize()} {paarts[1].lower()}"
                        #ic(local_name)
                    else:
                        local_name = paarts[0].capitalize()
                        #ic(local_name)


        all_trees.extend(tree_status_list)
        #ic(all_trees)

        grass_status_list = []
        """---------------GRASS_ESTABLISHMENT------------"""
        #get all grasses 
        if x["tree_contour-planted_choice5"] in ["both", "grasses"]:
            grass_species_list = str(x["grass_contour-contour_grass_species"]).split()
            #default
            scientific_name = ""
            local_name = ""

            for idx2 ,raw in enumerate(grass_species_list):
                #case 1: others
                if raw.strip() == 'other':
                    other_raw = str(x["grass_contour-other_contours_grass"])

                    #split by comma & 'and'
                    names = re.split(r",| and ", other_raw, flags=re.IGNORECASE)

                    #loop through local names
                    for idx, name in enumerate(names):
                        cleaned = name.strip()
                        if cleaned != 'nan':
                            grass_current_status_1 = ({
                                "id": idx + 1,
                                "local_name": cleaned,
                                "scientific_name": "",
                                "econtrol_status_id": 1
                            })
                        grass_status_list.append(grass_current_status_1)

                    continue
                
                #case2: ends with other and also has nan in other colun
                elif raw.strip().endswith('other'):
                    #use the 'other' column
                    local_name = str(j['halfmoon_grasses-other_halfmoon_grass']).strip()
                #case 3: split local name from scientific name
                else:
                    #split scientific and local names
                    if "(" in raw and ")" in raw:
                        #scientific name cleaning
                        parts = raw.split("(")[0].strip().replace("_", " ").split()
                        if len(parts) == 2:
                            scientific_name = f"{parts[0].capitalize()} {parts[1].lower()}"
                        else:
                            scientific_name = raw.split("(")[0].strip().replace("_", " ")

                        #local name cleaning
                        paarts = raw.split("(")[1].split(")")[0].strip().replace("_", " ").split()
                        if len(paarts) == 2:
                            local_name = f"{paarts[0].capitalize()} {paarts[1].lower()}"
                        else:
                            local_name = raw.split("(")[1].split(")")[0].strip().replace("_", " ").replace(",", " ")#[1].lower()
                    else:
                        scientific_name = raw.strip()

                if local_name != 'nan':
                    grass_current_status = {
                        "id": idx2 + 1,
                        "local_name": local_name,
                        "scientific_name": scientific_name,
                        "econtrol_status_id": 1
                    }
            
                #append the current status for ths record to the empty list
                grass_status_list.append(grass_current_status)
        
        """--------------------CURRENT STATUS--------------------"""
        idxx += 1
        currentStatus= {
            "id": idxx,
            "key":c["KEY"],            
            "herbaceous_cover": "",
            "visible_erosion": True,
            "erosion_type": [
                "OTHER"
            ],
            "other_erosion_type": "NONE",
            "is_intervention_effective": True,
            "longitude": x["gps-Longitude"],
            "latitude": x["gps-Latitude"],
            "altitude": x["gps-Altitude"],
            "accuracy": x["gps-Accuracy"],
            "photo_url": x["photo"],
            "comments": f"agric_field_state {x["managements_contour-agricultural_field_state"] if pd.notna(x["managements_contour-agricultural_field_state"]) else "MIGRATED_NOT_KNOWN"} state {c["geography-state_name"].replace("â€¦", "").replace(".", "").replace("..", "")} community {c['geography-community_name']}",
            "econtrolId": 1,
            "grassCurrentStatus":grass_status_list,
            "treeCurrentStatus":tree_status_list
        }
        
        #add every status to the to serve as establishment        
        all_grasses.extend(grass_status_list)

        cureent_status_list.append(currentStatus)


    """-----------------------------------contours DETAILS-------------------------------------"""

    contours_details = {
        "id": idx + 1,
        "key":c["KEY"],
        "erosion_control_type": "Contours",
        "established_date": datetime.strptime(c['contour-contour_details-date'], "%d/%m/%Y").strftime("%Y-%m"),
        "material_used": [
            "OTHER"
        ],
        "other_material_used": "MIGRATED_NOT_KNOWN",
        "who_established": ["MALE", "FEMALE"] if c["contour-contour_details-manages"] == 'both' else "MALE" if c["contour-contour_details-manages"]=="male" else "",
        "other_who_established": "YOUTH" if c["contour-contour_details-youth_manages"] == 'yes' else "",
        "total_interventions": c["contour-contour_details-total"],
        "length": c["contour-contour_details-length"],
        "width": c["contour-contour_details-width"],
        "depth": c["contour-contour_details-depth"],
        "vertical_spacing": 0,
        "horizontal_spacing": 0,
        "labor_payment": c["contour-contour_details-paid_unpaid_labor"],        
        "has_grass": any(grass_present),
        "total_number_grasses_planted": total_num_grasses_planted,
        "kg_grass_seeds_planted": total_kg_grass_seeds,
        "has_trees": any(trees_present),
        "total_number_different_trees_planted": total_num_tree_species,
        "total_number_trees_planted": total_num_trees_planted,
        "total_number_trees_survived": total_num_trees_survived,
        "management_practices":  [v for k, v in mapping_dict.items() if k in mngmts] + (["OTHER"] if any(k not in mapping_dict.keys() for k in mngmts) else []),
        "other_managements": " ".join([c for c in mngmts if c not in mapping_dict.keys()]),
        "usages":["MIGRATED_NOT_KNOWN"],
        "other_usages": "",
        "rangelandEntryId": 2,
        "currentStatus": cureent_status_list,
        "grassEstablishment": all_grasses,
        "treeEstablishment":all_trees   
    }

    contours_list.append(contours_details)
    


contours_econtrol = [{"econtrol": [i], "key" :i["key"]} for i in contours_list]


ic| trees_species_list: ['other']


TypeError: can only concatenate str (not "int") to str

In [156]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(contours_econtrol, f, indent=4)

## merge components into one json

In [157]:
# component lists
lists = [farming_entity_list, contours_econtrol, plot_list, projects_list]

# The output dictionary
merged = {}

for lst in lists:
    ic(len(lst))
    for item in lst:
        k = item['key']

        #check if already updated
        if k not in merged:
            merged[k] = {}
            
        #merge the item into the existing dictionary
        merged[k].update(item)

final = list(merged.values())
for i in final:
    #match enumerators
    for l in enums:
        if i['key'] == l['key']:
            i['enumerator'] = l


    i["boma"] = []
    i["cgrazing"] = []
    i["microcatchment"]= []
    i["seedbank"] = []
    i["seeding"] = []
    i["waterpoint"] = []
    i["completed"] = True
    i["rangeModuleId"] = 1
    i["currentStep"]= "DONE"
    i["moduleId"] = 6
    i["is_revisit"] = False
    #get the date_collected
    for j in records:
        if i["key"] == j["KEY"]:
            i["date_collected"] = datetime.strptime(j['basic_info-date'], "%d/%m/%Y").strftime("%Y-%m-%d")
    #match project id
    for x in projects_list:
        if i['key'] == x['key']:
            i['projectId'] = x['project']['id']
    #match plot id
    for k in plot_list:
        if i['key'] == k['key']:
            i['plotId'] = k['plot']['id']
    #match farming_entity_id
    for h in farming_entity_list:
        if i['key'] == h['key']:
            i['farmingEntityId'] = h['farmingEntity']['id']
    #match enumerator id
    for e in enums:
        if i['key'] == e['key']:
            i['enumeratorId'] = e['id']


ic(len(final))


ic| len(lst): 9
ic| len(lst): 9
ic| len(lst): 9
ic| len(lst): 9
ic| len(final): 9


9

In [158]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(final, f, indent=4)

## clean up keys

In [159]:
#clean up keys
def remove_keys(obj, key="key"):
    if isinstance(obj, dict):
        obj.pop(key, None)
        for v in obj.values():
            remove_keys(v, key)
    elif isinstance(obj, list):
        for v in obj:
            remove_keys(v, key)
    return obj

finaal = [x for x in final if x['key'] == 'uuid:b4a2d545-66fe-4d23-825b-41025149b460']
finaal = [remove_keys(i) for i in final]

#add id
for idx, c in enumerate(finaal):
    c['id'] = idx + 1



In [160]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(finaal, f, indent=4)